<a href="https://colab.research.google.com/github/nangokosu/ebay_ml/blob/main/EfficientNet_Siamese_Cosine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.2

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.2`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [ ]:
import keras
from keras.layers import *
import numpy as np
import tensorflow_addons as tfa
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from skimage import color,data,transform,filters,util,restoration
import tensorflow as tf
from keras.models import Model

In [ ]:
!pip install tf-nightly

     |████████████████████████████████| 399.4MB 40kB/s 
     |████████████████████████████████| 12.2MB 41.6MB/s 
     |████████████████████████████████| 4.0MB 44.0MB/s 
     |████████████████████████████████| 4.0MB 48.5MB/s 
     |████████████████████████████████| 471kB 45.8MB/s 
ERROR: tensorflow 2.4.0 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.0 which is incompatible.
ERROR: tensorflow 2.4.0 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install efficientnet

     |████████████████████████████████| 51kB 2.9MB/s 


In [ ]:
#import tensorflow as tf
import efficientnet.keras as efn 

In [ ]:
import tensorflow.keras.backend as K
def squeeze_excite_block(tensor, ratio=16):
    init = tensor
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)
    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)
    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)
    x = Multiply()([init, se])
    return x

In [ ]:
def create_encoder(input_shape):
  inputs=Input(shape=input_shape)
  enet=efn.EfficientNetB2(include_top=False,weights='imagenet',input_tensor=inputs,pooling=None)
  enet.trainable=False
  outputs=squeeze_excite_block(enet.output,ratio=16)
  outputs=GlobalAveragePooling2D()(outputs)
  outputs=Dropout(0.2)(outputs)
  outputs=Dense(120,activation="relu")(outputs)
  outputs=Dense(50,use_bias=False)(outputs)
  #outputs=BatchNormalization()(outputs)
  model = Model(
      inputs=inputs, outputs=outputs, name='encoder'
    )
  return model

In [ ]:
shape=(32,32,3)
input_1=Input(shape=(32,32,3))
input_2=Input(shape=(32,32,3))
encoder=create_encoder(shape)

31940608/31936256 [==============================] - 1s 0us/step


In [ ]:
# ONLY APPLIES FOR RETRAINING
# encoder=tf.keras.load_model("/content/drive/Shareddrives/eBay ML/EfficientNet__siamese_encoder")

In [ ]:
encoded_1=encoder(input_1)
encoded_2=encoder(input_2)

In [ ]:
def cosine_distance(encoded_no1,encoded_no2):
  predicted_sim=Dot(axes=1,normalize=True)([encoded_1,encoded_2])
  return predicted_sim

In [ ]:
predicted_sim=cosine_distance(encoded_1,encoded_2)
sim_model=Model([input_1,input_2],predicted_sim)

In [ ]:
sim_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
encoder (Functional)            (None, 50)           8191450     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dot (Dot)                       (None, 1)            0           encoder[0][0]                

In [ ]:
lrplateau1=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.20,patience=8)

In [ ]:
early_stop=tf.keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

In [ ]:
modelCheckpoint=tf.keras.callbacks.ModelCheckpoint('/content/drive/Shareddrives/eBay ML/EfficientNet_Res_Cosine',save_best_only=True)
# Might be saved in my own drive right now; check to be sure

In [ ]:
# Only run this if you are RETRAINING
# sim_model=tf.keras.models.load_model('/content/drive/Shareddrives/eBay ML/EfficientNet_Res_Cosine')
# If you are RETRAINING, CHANGE UP THE CODE BLOCK BELOW TO COVER A DIFFERENT QUARTER OF THE TRAINING DATASET.

In [ ]:
sim_model.compile(optimizer="Adam",loss="mean_squared_error")

In [ ]:
df = pd.read_pickle("/content/drive/Shareddrives/eBay ML/normalized_df_pairs_balanced.pkl")
length = df.shape[0]
# 2nd Quarter
df = df.iloc[length // 4: 2 * length // 4]    # change this QUARTER FOR RETRAINING
df["similarity"]=df["similarity"].astype("int32")
##df["image"]=df.input_tup.apply(lambda x: x[0])
#df['label']=df.input_tup.apply(lambda x: x[1])
#labels=pd.get_dummies(df.label)
# -1 is not similar

In [ ]:
#from collections import Counter
#Counter(df.similarity)

Counter({-1: 49265, 1: 49312})

In [ ]:
image_1=efn.preprocess_input(np.stack(df["image_1"]))
image_2=efn.preprocess_input(np.stack(df["image_2"]))

In [ ]:
#tf.keras.callbacks.ReduceLROnPlateau

In [ ]:
sim_model.fit([image_1,image_2],df.similarity.values,callbacks=[modelCheckpoint,early_stop,lrplateau1],validation_split=0.10,epochs=60,batch_size=64)

Epoch 1/60
1387/1387 [==============================] - 387s 268ms/step - loss: 0.8837 - val_loss: 0.9373
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/EfficientNet_Res_Cosine/assets
Epoch 2/60
1387/1387 [==============================] - 367s 264ms/step - loss: 0.8265 - val_loss: 0.9190
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/EfficientNet_Res_Cosine/assets
Epoch 3/60
1387/1387 [==============================] - 365s 263ms/step - loss: 0.8124 - val_loss: 0.9127
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/EfficientNet_Res_Cosine/assets
Epoch 4/60
1387/1387 [==============================] - 362s 261ms/step - loss: 0.8097 - val_loss: 0.9143
Epoch 5/60
1387/1387 [==============================] - 363s 262ms/step - loss: 0.8053 - val_loss: 0.9203
Epoch 6/60
1387/1387 [==============================] - 361s 260ms/step - loss: 0.8069 - val_loss: 0.9098
INFO:tensorflow:Assets written to: /content/drive/Shareddr

In [ ]:
tf.keras.models.save_model(encoder,"/content/drive/Shareddrives/eBay ML/EfficientNet__siamese_encoder")

INFO:tensorflow:Assets written to: /content/drive/Shareddrives/eBay ML/EfficientNet__siamese_encoder/assets


In [ ]:
df.similarity.values

array([-1,  1, -1, ..., -1, -1,  1], dtype=int32)